In [1]:
import cv2
import numpy as np

def load_gray_tiff(path: str) -> np.ndarray:
    """
    Load 8-bit grayscale TIFF using OpenCV
    """
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Gagal load image: {path}")
    return img.astype(np.float32)


def mask_valid(*imgs, min_valid=2):
    mask = np.ones_like(imgs[0], dtype=bool)
    for im in imgs:
        mask &= (im >= float(min_valid))
    return mask


def estimate_uter(deltaU, phi_high, mask, exclude_top_percent=0.1):
    """
    Estimate (Uter - Ulow) from brightest region
    excluding top 0.1% intensities
    """
    safe = mask & np.isfinite(deltaU)
    vals = phi_high[safe]

    p_hi = 100.0 - exclude_top_percent
    p_lo = p_hi - 0.4

    thr_hi = np.percentile(vals, p_hi)
    thr_lo = np.percentile(vals, p_lo)

    band = safe & (phi_high >= thr_lo) & (phi_high <= thr_hi)

    if np.sum(band) == 0:
        return float(np.nanmax(deltaU[safe]))

    return float(np.nanmedian(deltaU[band]))


def rs_from_ratio_cv2(phi_low_path,
                      phi_high_path,
                      Uth,
                      J_inj,
                      min_valid=2,
                      exclude_top_percent=0.1):

    phi_low  = load_gray_tiff(phi_low_path)
    phi_high = load_gray_tiff(phi_high_path)

    mask = mask_valid(phi_low, phi_high, min_valid=min_valid) \
           & (phi_low > 0) & (phi_high > 0)

    deltaU = np.full_like(phi_low, np.nan, dtype=np.float32)
    deltaU[mask] = Uth * np.log(phi_high[mask] / phi_low[mask])

    Uter_minus_Ulow = estimate_uter(deltaU, phi_high, mask, exclude_top_percent)

    Rs = np.full_like(deltaU, np.nan, dtype=np.float32)
    Rs[mask] = (Uter_minus_Ulow - deltaU[mask]) / float(J_inj)

    # c(r) relatif
    c_rel = np.full_like(phi_low, np.nan, dtype=np.float32)
    c_rel[mask] = phi_low[mask] / np.nanmedian(phi_low[mask])

    return {
        "Rs_map": Rs,
        "deltaU": deltaU,
        "Uter_minus_Ulow": Uter_minus_Ulow,
        "c_rel": c_rel
    }

In [3]:
result = rs_from_ratio_cv2(
    phi_low_path = r"C:\Users\Ghozy Abror\OneDrive - Institut Teknologi Bandung\Karirku\UNSW\Thesis\Coding\EL_Cell\10084_35_1_01272020_20\10084_35_1_01272020_20_cell_000.tiff",
    phi_high_path = r"C:\Users\Ghozy Abror\OneDrive - Institut Teknologi Bandung\Karirku\UNSW\Thesis\Coding\EL_Cell\10084_35_1_01272020_80\10084_35_1_01272020_80_cell_000.tiff",
    Uth=0.0259,        # bisa treat sebagai effective parameter
    J_inj=0.05         # A/cm² (I_inj / area_cell)
)

print("Median Rs:", np.nanmedian(result["Rs_map"]))

Median Rs: 0.15358216
